<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Оценка стоимости автомобилей на вторичном рынке

Многие знают про маркетплейсы где продаются б/у вещи, на которых есть возможность недорого купить качественную и полезную вещь. Но всегда волнует вопрос - кто и как устанавливает цену, и какие его характеристики больше всего влияют на итоговую стоимость продажи?! Вопрос становиться особо актуальным, если речь идет про дорогие товары, например про автомобили!
Целью данного проекта будет разработка модели предсказания стоимости автомобиля на вторичном рынке.

In [1]:
import pandas as pd
import os
from sklearn.preprocessing import OneHotEncoder
from joblib import parallel_backend
from sklearn.model_selection import train_test_split

In [2]:
from tqdm import tqdm
from tqdm import trange
from sklearn.inspection import permutation_importance

In [3]:
import winsound
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV

In [4]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn

In [5]:
import pandas_profiling

C:\Users\Admin\anaconda3\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
C:\Users\Admin\AppData\Roaming\Python\Python310\site-packages\visions\backends\shared\nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:
C:\Users\Admin\AppData\Local\Temp\ipykernel_17652\1591302161.py:1: DeprecationWarning: `import pandas_profiling` is going to be de

In [6]:
from sklearn.dummy import DummyClassifier
import time
import datetime
from sklearn.metrics import mean_absolute_percentage_error

In [7]:
from catboost import CatBoostRegressor 
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import ExtraTreeRegressor

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegressionCV

In [9]:
import re
import requests,json

In [10]:
!pip install -q pyod
!pip install -q feature_engine
!pip install -q imbalanced-learn

In [11]:
import warnings
warnings.filterwarnings('ignore')
from collections import Counter

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams, rcParamsDefault

Намстройка параметров визуализаций

In [12]:
rcParams['figure.figsize'] = 8,8
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline
factor = 1
default_dpi = rcParamsDefault['figure.dpi']
rcParams['figure.dpi'] = default_dpi*factor

pd.options.display.max_colwidth = 50
pd.options.display.max_columns = 50

In [13]:
# важно! from imblearn.pipeline import make_pipeline (не из sklearn.pipeline import make_pipeline)

from sklearn.experimental import enable_iterative_imputer

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.impute import KNNImputer, IterativeImputer, SimpleImputer
from sklearn.metrics import f1_score

from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import PolynomialFeatures

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.compose import ColumnTransformer, make_column_selector, make_column_transformer

from imblearn.combine import SMOTETomek
from imblearn.base import FunctionSampler
from imblearn.pipeline import make_pipeline as make_imblearn_pipeline

from feature_engine.selection import DropCorrelatedFeatures, DropConstantFeatures


In [14]:
#Это путь для загурзки в среде Практикума
pth1_train = '/datasets/hotel_train.csv'
pth1_test = '/datasets/hotel_test.csv'
pth2_train = 'E://Data//_Projects//kaggle//auto//train.csv'
pth2_test = 'E://Data//_Projects//kaggle//auto//test.csv'

if os.path.exists(pth1_train) and os.path.exists(pth1_test):
    df = pd.read_csv(pth1_train)
    dft = pd.read_csv(pth1_test)
elif os.path.exists(pth2_train) and os.path.exists(pth2_test):
    df = pd.read_csv(pth2_train, index_col=False)
    dft = pd.read_csv(pth2_test, index_col=False)
else:
    print('Something is wrong')


# Отрежем часть датасета, чтобы быстро всё считалось, когда будет готово -уберём !!!

In [15]:
df=df.sample(4400)
df.head(4)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,sellingprice,saledate
284170,2008,HUMMER,H3,Base,SUV,automatic,5gten13e888151081,ca,4.0,62825.0,gray,black,power drive inc,17250,Wed Feb 11 2015 04:15:00 GMT-0800 (PST)
254546,2013,Ford,Explorer,XLT,SUV,automatic,1fm5k7d84dga30846,ca,4.2,23362.0,white,black,orange county's credit union,26250,Wed Jan 07 2015 12:05:00 GMT-0800 (PST)
50387,2013,Hyundai,Elantra,GLS,Sedan,automatic,5npdh4ae4dh200562,oh,2.7,63705.0,gray,tan,the hertz corporation,9500,Tue Feb 03 2015 01:30:00 GMT-0800 (PST)
5957,2011,Ford,Edge,SEL,SUV,automatic,2fmdk4jc0bbb51879,ca,3.3,43357.0,black,black,"ford motor credit company,llc pd",19000,Thu Dec 18 2014 12:00:00 GMT-0800 (PST)


# Отрежем часть датасета, чтобы быстро всё считалось, когда будет готово -уберём !!!

In [16]:
df['saledate'] = pd.to_datetime(df['saledate'],utc=True)
df['year_sale'] = df['saledate'].dt.year
df['mnth_sale'] = df['saledate'].dt.month
X = df.drop(columns='sellingprice')
y = df.sellingprice

cat_colunmns = ["year","make","model", "trim","body","transmission","state","condition","color","interior",'year_sale','mnth_sale',"vin",'seller','saledate']
num_columns = ['odometer']

In [17]:
def outlier_detector_(X, y):
    outlier_index =  IsolationForest(contamination=0.01, random_state=42).fit_predict(X)
    return X[outlier_index == 1], y[outlier_index == 1] 

In [18]:
def small_scripter_(X):
    print("Setting strings to lower case")
    for i in cat_colunmns:
        try:
            X[i]=X[i].astype(str)
            X[i] = [x.lower() for x in X[i]]
        except:
            print("A NaN encountered")
    return X

In [19]:
body_dic={'nan':None, 'incomplete - cutaway':None,
          "extended cab":"cab",'regular cab':'cab',
          'crew cab':'cab','supercab':'cab','access cab':'cab',
          'quad cab':'cab','king cab':'cab','double cab':'cab',
          'crewmax cab':'cab','club cab':'cab',
          'mega cab':'cab','regular-cab':'cab','xtracab':'cab',
          'cab plus 4':'cab','cab plus':'cab',
          'sport utility vehicle (suv)/multi-purpose vehicle (mpv)':'suv',
          'g coupe':'coupe','g sedan':'sedan','e-series van':'van',
          'minivan':'van','sedan/saloon':'sedan','cargo van':'van',
          'cts coupe':'coupe', 'elantra coupe':'coupe',
          'hatchback/liftback/notchback':'hatchback','convertible/cabriolet':'cab',
           'genesis coupe': 'coupe','incomplete':None, 'g37 coupe':'coupe',
           'q60 coupe':'coupe','promaster cargo van':'van','cts-v coupe':'coupe',
          'transit van':'van','g convertible':'convertible','g37 convertible':'convertible',
          'q60 convertible':'convertible','ram van':'van','granturismo convertible':'convertible',
          'cts-v wagon':'wagon', 'tsx sport wagon': 'wagon',
          'cts wagon':'wagon','koup':'coupe','beetle convertible':'convertible'
         }
trans_dic={'':None,'none':None,'nan':None,'manual/standard':'manual'}
interior_dic={'—':None,'nan':None}
color_dic={'—':None,'nan':None}
make_dic={'nan':None,'dodge tk':'dodge','mazda tk':'mazda', 'mercedes-benz': 'mercedes',
          'mercedes-b':'mercedes','volkswagen':'vw','ford truck':'ford','gmc truck':'gmc'}

In [74]:
def replace_duplicates_(X):
    dfs=pf.DataFrame(X,columns=list(df.columns))
    print("Replacing dupes")
    dfs['body']=dfs['body'].replace(body_dic)
    dfs['transmission']=dfs['transmission'].replace(trans_dic)
    dfs['interior']=dfs['interior'].replace(interior_dic)
    dfs['color']=dfs['color'].replace(color_dic)
    dfs['make']=dfs['make'].replace(make_dic)
    
    return dfs

In [62]:
def vin_extractor(vin_list):
    url = 'https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVINValuesBatch/';
    res=[]
    batchs=round(len(vin_list)/50)
    for i in range(batchs):
        try:
            l=(list(vin_list.iloc[i*50:(i+1)*50]))
            s = ';'.join(l)
            post_fields = {'format': 'json', 'data':s};
            r = requests.post(url, data=post_fields);
            res.append(r.text)
            winsound.Beep(440, 750)
            print("Vins from ", i*50, "to ",(i+1)*50, "appended.")
        except:
            print("Something gone wrong")
            winsound.Beep(640, 350)
        with open('res.json', 'w') as f:
            json.dump(res, f)

In [63]:
def loader_(X):
    #Делаем список винов, по которым есть пропуски
    for i in X.columns:
        X[i][X[i].apply(lambda i: True if re.search('^\s*$', str(i)) else False)]=None
    vins = X[X.isnull().values.any(axis=1)]['vin']
        
    #Проверяем, качали ли мы дынне из базы ранее
    if os.path.exists('res.json'):
        print("Restoring VIN-data from dump.")
    else:
        #Если не качали, то скачиваем через API
        print("Extracting VIN-data from vpic.nhtsa.dot.gov")
        vin_extractor(vins)
    with open('res.json') as f:
        d = json.load(f)
    out=pd.DataFrame(d)
    out1=pd.DataFrame()
    print("Parcing external source")
    for i in trange(len(out)):
        try:
            original_string = out.iloc[i].values[0]
            #define string
            #create pandas DataFrame from string
            pr = json.loads(original_string)
            pr1=pd.DataFrame(pr['Results'])
            out1=pd.concat([out1, pr1])
        except:
            print(i,"-Failed")
    winsound.Beep(640, 350)
    return out1

In [64]:
out1=loader_(X)

Restoring VIN-data from dump.
Parcing external source


100%|██████████████████████████████████████████████████████████████████████████████| 1363/1363 [00:53<00:00, 25.39it/s]


In [65]:
def gaps_filler_from_api_(X):

 
    pbar = tqdm(out1.columns, ncols=80)
    print("Finding NaNs")
    for i in pbar:
    #Заменим все пустые и None значения на настоящий None
        out1[i][out1[i].apply(lambda i: True if re.search('^\s*$', str(i)) else False)]=None
        out1[i][out1[i].apply(lambda i: True if re.search('.*None.*', str(i)) else False)]=None
    alias={'make':'Make', 'model':'Model','trim':'Trim','body':'BodyClass','transmission':'TransmissionStyle','year':'ModelYear'}
    pbar = tqdm(list(alias), ncols=50)
    print("Filling gaps from external source")
    for i in pbar:
        listOfVins=list(X.loc[X[i].isna()==True]['vin'])
        listOfInt=list(out1[out1['VIN'].isin(listOfVins)][alias[i]])
        dictOfWords = dict(zip(listOfVins, listOfInt))
        X[i]= X[i].fillna(X['vin'].map(dictOfWords))
    small_scripter_(X)
    return X

In [66]:
def drop_extras_(X):
    print("Dropping extra columns")
    X=X.drop(['seller','saledate','vin','trim'], axis=1)
    print("Encoding...")
    return X

In [67]:
from sklearn.preprocessing import FunctionTransformer
outlier_detector = FunctionTransformer(outlier_detector_)
small_scripter = FunctionTransformer(small_scripter_)
duplicate_replacer = FunctionTransformer(replace_duplicates_)
api_gaps_filler = FunctionTransformer(gaps_filler_from_api_)
extra_cols_dropper = FunctionTransformer(drop_extras_)

In [87]:
#num_pipeline = make_imblearn_pipeline(StandardScaler(), PolynomialFeatures(), DropCorrelatedFeatures(threshold=0.9), DropConstantFeatures(),)
column_transformer = make_column_transformer(#(num_pipeline, num_columns),
                                             (api_gaps_filler,cat_colunmns),
                                             (duplicate_replacer,cat_colunmns),
                                             #(extra_cols_dropper,cat_colunmns),
                                             #(OneHotEncoder(drop='first', sparse=False), cat_colunmns),
                                             remainder='passthrough',n_jobs=16,verbose=5,verbose_feature_names_out=False)
column_transformer

ColumnTransformer(n_jobs=16, remainder='passthrough',
                  transformers=[('functiontransformer-1',
                                 FunctionTransformer(func=<function gaps_filler_from_api_ at 0x000001790154F5B0>),
                                 ['year', 'make', 'model', 'trim', 'body',
                                  'transmission', 'state', 'condition', 'color',
                                  'interior', 'year_sale', 'mnth_sale', 'vin',
                                  'seller', 'saledate']),
                                ('functiontransformer-2',
                                 FunctionTransformer(func=<function replace_duplicates_ at 0x00000178E5CDF010>),
                                 ['year', 'make', 'model', 'trim', 'body',
                                  'transmission', 'state', 'condition', 'color',
                                  'interior', 'year_sale', 'mnth_sale', 'vin',
                                  'seller', 'saledate'])],
                  verbose=5, verbose_feature_names_out=False)

In [88]:
pipeline = make_imblearn_pipeline(column_transformer) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xx=pipeline.fit_transform(X_train,y_train)
print("Complete")
winsound.Beep(640, 350)

Complete


In [120]:
a = "LC(Carbamidomethyl)RLK"
re.sub('(.*)\((.+)\)(.*)', '\g<2>', a)

'Carbamidomethyl'

In [108]:
column_transformer.output_indices_

{'functiontransformer-1': slice(0, 15, None),
 'functiontransformer-2': slice(15, 30, None),
 'remainder': slice(30, 31, None)}

In [129]:
l=str(list(column_transformer.output_indices_.items())[0][1])
rel=re.sub('(.*)\((.+)\)(.*)', '\g<2>', l)
rel_list=rel.split(',')
int(rel_list[0]), int(rel_list[1])

(0, 15)

In [110]:
slcs=pd.DataFrame(column_transformer.output_indices_.items(),columns=['layer','slice'])
slcs

,layer,slice
0,functiontransformer-1,"slice(0, 15, None)"
1,functiontransformer-2,"slice(15, 30, None)"
2,remainder,"slice(30, 31, None)"


In [90]:
pdxx=pd.DataFrame(xx)
pd.set_option('display.max_columns', None)
display(pdxx.head(50))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
0,2008,chevrolet,silverado 2500hd,work truck,extended cab,automatic,wi,1.9,white,gray,2015,2,1gchc29k78e138031,fleet lease disposal,2015-02-17 18:00:00+00:00,2008,Chevrolet,Silverado 2500HD,Work Truck,Extended Cab,automatic,wi,1.9,white,gray,2015,2,1gchc29k78e138031,fleet lease disposal,2015-02-17 18:00:00+00:00,235547.0
1,2012,jeep,wrangler,unlimited sahara,suv,none,on,4.0,black,brown,2015,1,1c4bjweg0cl120575,repo depo,2015-01-26 18:00:00+00:00,2012,Jeep,Wrangler,Unlimited Sahara,SUV,NaN,on,4.0,black,brown,2015,1,1c4bjweg0cl120575,repo depo,2015-01-26 18:00:00+00:00,78250.0
2,2012,nissan,altima,2.5 s,sedan,automatic,ga,4.4,white,black,2015,1,1n4al2ap8cc106420,nissan-infiniti lt,2015-01-26 17:30:00+00:00,2012,Nissan,Altima,2.5 S,Sedan,automatic,ga,4.4,white,black,2015,1,1n4al2ap8cc106420,nissan-infiniti lt,2015-01-26 17:30:00+00:00,23011.0
3,2013,mercedes-benz,c-class,c350 sport,sedan,automatic,ca,3.8,none,black,2015,2,wddgf5hb9dr255673,mercedes-benz financial services,2015-02-03 20:30:00+00:00,2013,Mercedes-Benz,C-Class,C350 Sport,Sedan,automatic,ca,3.8,None,black,2015,2,wddgf5hb9dr255673,mercedes-benz financial services,2015-02-03 20:30:00+00:00,26021.0
4,2012,lexus,gx 460,base,suv,automatic,mo,3.2,black,black,2015,6,jtjbm7fx3c5045061,molle volkswagen molle audi,2015-06-16 20:30:00+00:00,2012,Lexus,GX 460,Base,SUV,automatic,mo,3.2,black,black,2015,6,jtjbm7fx3c5045061,molle volkswagen molle audi,2015-06-16 20:30:00+00:00,48523.0
5,2013,nissan,altima,2.5,sedan,automatic,ca,3.7,gray,black,2015,2,1n4al3apxdc171146,nissan-infiniti lt,2015-02-02 20:30:00+00:00,2013,Nissan,Altima,2.5,Sedan,automatic,ca,3.7,gray,black,2015,2,1n4al3apxdc171146,nissan-infiniti lt,2015-02-02 20:30:00+00:00,23867.0
6,2005,ford,e-150,none,van,automatic,tx,5.0,white,gray,2015,2,jtjht00w754001741,pm standley motorcars,2015-02-03 18:20:00+00:00,2005,NaN,NaN,NaN,NaN,automatic,tx,5.0,white,gray,2015,2,jtjht00w754001741,pm standley motorcars,2015-02-03 18:20:00+00:00,125830.0
7,2014,ford,f-350 super duty,lariat,crew cab,automatic,fl,4.2,blue,beige,2015,2,1ft8w3dt8eea28128,tdaf remarketing,2015-02-02 17:00:00+00:00,2014,Ford,F-350 Super Duty,Lariat,Crew Cab,automatic,fl,4.2,blue,beige,2015,2,1ft8w3dt8eea28128,tdaf remarketing,2015-02-02 17:00:00+00:00,44813.0
8,2012,nissan,murano,sl,suv,automatic,oh,4.6,blue,black,2015,1,jn8az1mw1cw230635,wholesale inc # 2,2015-01-13 01:30:00+00:00,2012,Nissan,Murano,SL,SUV,automatic,oh,4.6,blue,black,2015,1,jn8az1mw1cw230635,wholesale inc # 2,2015-01-13 01:30:00+00:00,31534.0
9,2014,ford,escape,titanium,suv,automatic,il,3.3,gray,gray,2014,12,1fmcu9j90eua79409,avis budget group,2014-12-18 02:00:00+00:00,2014,Ford,Escape,Titanium,SUV,automatic,il,3.3,gray,gray,2014,12,1fmcu9j90eua79409,avis budget group,2014-12-18 02:00:00+00:00,37586.0


In [ ]:
pipeline = make_imblearn_pipeline(column_transformer,                     
                         IterativeImputer(),
                         FunctionSampler(func=outlier_detector_),
                         SMOTETomek(),
                         RandomForestClassifier()) 

pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.head()

In [ ]:
X_train.shape, y_train.shape

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
pipeline.fit(X_train, y_train)

pipeline.predict(X_test)

pipeline.score(X_test, y_test)